In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

In [8]:
df = pd.read_csv('./data/dataset.csv')
print('data shape: {}'.format(df.shape))

X = df['NAME']
y = df['NATIONALITY']

classes = y.unique()
num_classes = len(y.unique())

print('number of classes: {}'.format(num_classes))

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, random_state=42)

print('train data shape X, y: {},{}'.format(X_train_df.shape, y_train_df.shape))
print('test data shape X, y: {},{}'.format(X_test_df.shape, y_test_df.shape))

data shape: (6826, 2)
number of classes: 6
train data shape X, y: (5460,),(5460,)
test data shape X, y: (1366,),(1366,)


In [9]:
X_tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
          lower=True, char_level=True, oov_token=None)

y_tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
          lower=True, char_level=False, oov_token=None)

In [10]:
X_train = X_train_df.values.astype(str) # Otherwise, there's an error when calling 'fit_on_texts' >> AttributeError: 'int' object has no attribute 'lower'
X_test = X_test_df.values.astype(str) # Otherwise, there's an error when calling 'fit_on_texts' >> AttributeError: 'int' object has no attribute 'lower'

X_tokenizer.fit_on_texts(X_train)
X_train = X_tokenizer.texts_to_sequences(X_train)
X_test = X_tokenizer.texts_to_sequences(X_test)

X_train = X_tokenizer.sequences_to_matrix(X_train)
X_test = X_tokenizer.sequences_to_matrix(X_test)

In [11]:
# encode from string labels to numerical labels 
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_df.values.astype(str)) # error without astype(str)
y_test = label_encoder.transform(y_test_df.values.astype(str))
# print(encoder.classes_.shape)
# encoder.inverse_transform(y_train)

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [12]:
len(y_train)

5460

In [13]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = len(X_tokenizer.word_counts)
batch_size = 32
maxlen = 20

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, output_dim=maxlen))
model.add(LSTM(maxlen, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)



5460 train sequences
1366 test sequences
Pad sequences (samples x time)
X_train shape: (5460, 15)
X_test shape: (1366, 15)
Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train...
Instructions for updating:
Use tf.cast instead.
Train on 5460 samples, validate on 1366 samples
Epoch 1/15
5460/5460 [==============================] - 5s 825us/step - loss: 1.5272 - acc: 0.2879 - val_loss: 1.3963 - val_acc: 0.3199
Epoch 2/15
5460/5460 [==============================] - 3s 565us/step - loss: 1.3923 - acc: 0.3284 - val_loss: 1.3908 - val_acc: 0.3199
Epoch 3/15
5460/5460 [==============================] - 3s 574us/step - loss: 1.3891 - acc: 0.3269 - val_loss: 1.3877 - val_acc: 0.3199
Epoch 4/15
5460/5460 [==============================] - 3s 576us/step - loss: 1.3876 - acc: 0.3282 - val_loss: 1.3865 - val_acc: 0.3199
Epoch 5/15
5460/5460 [====

In [ ]:
# df['lenght'] = df['NAME'].apply(lambda x: len(x))
# df['length']

In [ ]:
X_tokenizer